# 3.1 Performance Básico - Introducción a Optimización

**Resumen Ejecutivo**: Aprende a medir y mejorar el rendimiento de consultas SQL mediante análisis de planes de ejecución, estadísticas de I/O y patrones de optimización (sargability, proyección selectiva, índices lógicos).

---

## 🎯 Objetivos de Aprendizaje
- Medir rendimiento con `SET STATISTICS TIME/IO ON` y leer reads lógicos/físicos.
- Aplicar principios de sargability (filtros que aprovechan índices).
- Identificar consultas lentas y refactorizar para reducir tiempo y costo de CPU.

## 🧰 Prerrequisitos
- Haber completado Level02 y conocer JOINs, agregaciones, CTEs.
- Familiaridad conceptual con índices clustered/nonclustered.
- Acceso a herramientas de diagnóstico (SSMS o Azure Data Studio).

---

## 📚 Conceptos Clave
- **Tiempo de ejecución**: duración total de la query.
- **Reads lógicos**: páginas de 8KB leídas de memoria (buffer pool).
- **Reads físicos**: páginas leídas del disco (más lentas que memoria).
- **Sargability**: capacidad de un filtro para usar índices (`fecha >= '2024-01-01'` sí; `YEAR(fecha) = 2024` no).

> ⚠️ Consultas que funcionan con 10K registros pueden colapsar con 10M; medir siempre con volúmenes reales.

## Principios de optimización básica

### 1️⃣ Selecciona solo las columnas necesarias
❌ Malo: `SELECT * FROM tabla` → lee todas las columnas, más datos transferidos
✅ Bueno: `SELECT id, nombre FROM tabla` → lee solo lo necesario

### 2️⃣ Filtra lo más pronto posible
❌ Malo: hacer JOIN de 1M registros y luego filtrar con WHERE
✅ Bueno: filtrar en WHERE antes del JOIN (el optimizador ayuda pero sé explícito)

### 3️⃣ Usa índices apropiados
❌ Malo: `WHERE YEAR(fecha) = 2024` → función en columna impide uso de índice
✅ Bueno: `WHERE fecha >= '2024-01-01' AND fecha < '2025-01-01'` → sargable (index-friendly)

### 4️⃣ Evita funciones en columnas indexadas dentro de WHERE
Esto se llama **sargability** (Search ARGument ABLE).

In [ ]:
-- Activar estadísticas de tiempo y I/O
SET STATISTICS TIME ON;
SET STATISTICS IO ON;

-- Consulta de ejemplo: total de ventas por producto
SELECT p.producto_id, p.nombre, 
       SUM(v.cantidad) AS unidades_vendidas,
       SUM(v.cantidad * p.precio_unitario) AS ingresos_totales
FROM dbo.fact_ventas v
INNER JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
WHERE v.fecha >= '2024-01-01'
GROUP BY p.producto_id, p.nombre
ORDER BY ingresos_totales DESC;

-- Desactivar estadísticas
SET STATISTICS TIME OFF;
SET STATISTICS IO OFF;

/*
💡 Interpretación de resultados:
- SQL Server Execution Times: CPU time = XXms, elapsed time = XXms
- Table 'fact_ventas': Scan count X, logical reads XXX
- Si logical reads > 1000 por tabla, considera agregar índices
- Si elapsed time >> CPU time, puede haber esperas de I/O o locks
*/

Ejemplos de patrones sargables vs no sargables (conceptual).

## 🟢 Ejercicio Básico
Ejecuta esta consulta con STATISTICS IO ON y anota los logical reads:
```sql
SELECT cliente_id, COUNT(*) AS num_ventas
FROM dbo.fact_ventas
WHERE fecha >= '2024-01-01'
GROUP BY cliente_id;
```
Luego prueba cambiar el filtro a `WHERE YEAR(fecha) = 2024` y compara los reads. ¿Cuál es más eficiente?

## 🟠 Ejercicio Intermedio
Compara el performance de estas dos consultas:

**Opción A (subconsulta correlacionada):**
```sql
SELECT c.cliente_id, c.nombre,
       (SELECT SUM(v.cantidad) FROM fact_ventas v WHERE v.cliente_id = c.cliente_id) AS total
FROM dim_clientes c;
```

**Opción B (JOIN con agregación):**
```sql
SELECT c.cliente_id, c.nombre, ISNULL(SUM(v.cantidad), 0) AS total
FROM dim_clientes c
LEFT JOIN fact_ventas v ON c.cliente_id = v.cliente_id
GROUP BY c.cliente_id, c.nombre;
```

Mide CPU time y logical reads. ¿Cuál escala mejor con millones de registros?

## 🔴 Ejercicio Avanzado
Optimiza esta consulta problemática que tarda >10 segundos:

```sql
SELECT c.nombre, p.nombre AS producto, v.fecha, v.cantidad
FROM fact_ventas v
JOIN dim_clientes c ON v.cliente_id = c.cliente_id
JOIN dim_productos p ON v.producto_id = p.producto_id
WHERE UPPER(c.nombre) LIKE '%ACME%'
  AND MONTH(v.fecha) = 12
ORDER BY v.fecha DESC;
```

**Problemas a resolver:**
1. `UPPER(c.nombre)` en WHERE impide uso de índice
2. `MONTH(v.fecha)` no es sargable
3. No hay índices sugeridos

**Entrega:**
- Consulta refactorizada
- Índices recomendados (formato: `CREATE INDEX idx_nombre ON tabla(columnas)`)
- Justificación de cambios

## ⚙️ Desafío Senior
Diseña un benchmark completo:

1. Crea un SP que ejecute 5 consultas representativas de tu sistema
2. Mide tiempo promedio en 10 ejecuciones (descarta la 1ra por cache frío)
3. Propón 3 índices estratégicos
4. Vuelve a medir y calcula % de mejora
5. Documenta trade-offs (espacio en disco, impacto en INSERT/UPDATE)

**Entregable:** Script SQL con comentarios explicando cada decisión.

---

## Errores Comunes

❌ **SELECT * en producción**: lee columnas innecesarias, rompe cache de plan
❌ **No medir antes de optimizar**: "siento que está lento" no es dato
❌ **Funciones en WHERE indexado**: `WHERE UPPER(email)` destruye índice
❌ **Ignorar estadísticas de tabla**: SQL Server toma decisiones con stats desactualizadas
❌ **Índices en todo**: cada índice cuesta en INSERT/UPDATE, más espacio en disco

---

## Recursos
- [SQL Server Query Performance Tuning - Microsoft Learn](https://learn.microsoft.com/sql)
- [Sargable Predicates Explained](https://learn.microsoft.com)
- Siguiente: `02_indices_avanzados.ipynb` → tipos de índices (clustered, nonclustered, columnstore)

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[Siguiente ➡️](02_indices_avanzados.ipynb)
---
